# Transfer Learning Exercises

In [ ]:
# Import useful libraries        (note: don't forget to turn on GPU)

# tensorflow for network building/training
import tensorflow as tf
from tensorflow.python.keras import Model, Sequential
from tensorflow.keras.applications.resnet50 import ResNet50

# Basic operating system (os), numerical, and plotting functionality
import os
import numpy as np
from matplotlib import pylab as plt

# scikit-learn data utilities
from sklearn.model_selection import train_test_split

# Color transformations
from skimage.color import rgb2lab

#Skimage resizing 
from skimage.transform import resize

# Garbage collection (for saving RAM during training)
import gc

## ResNet50 Model

For this exercise you'll now use the ResNet50 model as the feature extractor. https://www.tensorflow.org/api_docs/python/tf/keras/applications/ResNet50

Specifications:
- Expected input size: 244x244 pixels
- Output classes: optional # of classes to classify images into, only to be specified if `include_top` is `True`, and if no weights argument is specified

Our images are 150x150 pixels in size and come from only 10 categories. In order to use this model for our classification task, we again need to do the following:
* Resize images : Our input images can be resized to the appropriate dimensions. Alternatively, we can pad our images to the expected dimensions. Padding leads to additional choices - Do we pad with zeros, duplicate edge pixels or mirror the image across edges?
* Change the prediction layer : Remove the existing prediction layer and add a new layer that can predict 8 classes.
* Train : Finally, we need to train the network on our data

## Load Data

***Note: for the exercise in this section you'll have to edit a line of code in a line of the cell for resizing images***

Getting path and changing directories

In [ ]:
# Define the current directory and the directory where the files to download can
# be found
current_dir = os.getcwd()
remote_path = 'https://github.com/BeaverWorksMedlytics2020/Data_Public/raw/master/NotebookExampleData/Week3/data_nuclei/crc/'

# Define and build a directory to save this data in
data_dir = os.path.join(current_dir, 'crc_data')
if not os.path.isdir(data_dir):
  os.mkdir(data_dir)

# Move into the data directory and download all of the files
os.chdir(data_dir)
for ii in range(1, 6):
    basename = f'rgb0{ii}.npz'
    filename = os.path.join(remote_path, basename)

    # Check if the file has already been downloaded
    if not os.path.isfile(basename):
      cmd = f'wget {filename}'
      print(cmd)
      os.system(cmd)

# Return to the original directory
os.chdir(current_dir)

Function for loading images

In [ ]:
# Define a function to load the data from the assumed download path
def load_images(colorspace='rgb'):
    """
    Loads the example data and applies transformation into requested colorspace

    Arguments
    ---------
    colorspace : str, optional, default: `rgb`
        The colorspace into which the images should be transformed. Accepted
        values include

        'rgb' : Standard red-green-blue color-space for digital images

        'gray' or 'grey': An arithmetic average of the (r, g, b) values

        'lab': The CIE L*a*b* colorspace
    
    Returns
    -------
    images : numpy.ndarray, shape (Nimg, Ny, Nx, Ncolor)
        The complete set of transformed images

    labels : numpy.ndarray, shape (Nimg)
        The classification labels associated with each entry in `images`

    label_to_str : dict
        A dictionary which converts the numerical classification value in
        `labels` into its string equivalent representation.
    """
    # Check that the colorspace argument is recognized
    colorspace_lower = colorspace.lower()
    if colorspace_lower not in ['rgb', 'gray', 'grey', 'lab']:
        raise ValueError(f'`colorspace` value of {colorspace} not recognized')

    # Load data, which is stored as a numpy archive file (.npz)
    filename = os.path.join(data_dir, 'rgb01.npz')
    print(f'loading {filename}')
    tmp = np.load(os.path.join(data_dir, 'rgb01.npz'), allow_pickle=True)

    # Parse the loaded data into images and labels
    # Initialize the images and labels variables using the first archive data
    images = tmp['rgb_data']
    if colorspace_lower == 'rgb':
        pass
    elif colorspace_lower in ['gray', 'grey']:
        images = np.mean(images, axis=-1)      # Average into grayscale
    elif colorspace_lower == 'lab':
        images = rgb2lab(images)               # Convert to CIE L*a*b*

    # Grab the initial array for the image labels
    labels = tmp['labels']
    
    # Grab the dictionary to convert numerical labels to their string equivalent
    label_to_str = tmp['label_str']
    label_to_str = label_to_str.tolist() # Convert label_to_str into a dict

    # Update the user on the number and size of images loaded
    print('Loaded images with shape {}'.format(images.shape))
    del tmp

    # Loop over each of the remaining archives and append the contained data
    for ii in range(2,6):
        # Build the full path to the archive and load it into memory
        filename = os.path.join(data_dir, f'rgb0{ii}.npz')
        print(f'loading {filename}')
        tmp = np.load(filename, allow_pickle=True)

        # Parse and append the data
        these_images = tmp['rgb_data']
        if colorspace_lower == 'rgb':
            pass
        elif (colorspace_lower == 'gray') or (colorspace_lower == 'grey'):
            these_images = np.mean(these_images, axis=-1) # Convert to grayscale
        elif colorspace_lower == 'lab':
            these_images = rgb2lab(these_images)          # Convert to CIEL*a*b*

        # Append the images and labels
        images = np.append(images, these_images, axis=0)
        labels = np.append(labels, tmp['labels'], axis=0)

        # Update the user on the number and size of images
        print('Loaded images with shape {}'.format(these_images.shape))
        del tmp

    # Force the image data to be floating point and print the data shape
    images = images.astype(np.float)
    print('Final image data shape: {}'.format(images.shape))
    print('Number of image labels: {}'.format(*labels.shape))

    return images, labels, label_to_str

Load images and labels into memory

In [ ]:
images, labels, label_to_str = load_images()

Resizing (remember we want 224x224 pixels)

In [ ]:
scale_images_bool = True
num_images = len(images)
if scale_images_bool:
    # Resize data
    ## YOUR CODE HERE
    for ii in range(0, num_images):
        if not np.mod(ii,1000):
            print(f'iter {ii}')
        resized_data[ii,::] = resize(images[ii,::], (224, 224, 3), mode='symmetric')
    images = resized_data
    resized_data = []
    print(images.shape)
    with open('resized_224.npz', 'wb') as fp:
        np.savez(fp, data=images, labels=labels)
else:
    print('no image scaling')

Split data into train and test sets

In [ ]:
#images.astype(np.float16)
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=.2)

# Print sizes of test/train sets
print(f'train_images.shape: {train_images.shape}')
print(f'train_labels.shape: {train_labels.shape}')
print(f'test_images.shape: {test_images.shape}')
print(f'test_labels.shape: {test_labels.shape}')

Scale image values to the range of 0->1 (if it hasn't been done already)

In [ ]:
# Note, we cast image data as float16 to save RAM
if train_images[0,::].max() >1:
    train_images = train_images.astype(np.float16)/255.
if test_images[0,::].max() >1:
    test_images = test_images.astype(np.float16)/255.

Converting 'labels' (1D array of integers) to onehot encoding

In [ ]:
# One hot pocket
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

print('onehot-encoded labels:')
print(train_labels)

## Load Pre-trained ResNet50 Model

In [ ]:
# Create the base pre-trained model
base_model = None
model = None
print('loading ResNet50')
base_model = ## YOUR CODE HERE
print('done')

Summarize model structure

In [ ]:
base_model.summary()

Modify the pre-trained ResNet50 network by adding a few new layers at the output, including a classification layer (remember we want to predict 8 different classes)

In [ ]:
output = base_model.output

# Add a global spatial average pooling layer
## YOUR CODE HERE

# Add a fully-connected layer
## YOUR CODE HERE

# Add the final classification layer
predictions = tf.keras.layers.Dense(## YOUR CODE HERE)

# Make the model you will train
model = Model(inputs=base_model.input, outputs=predictions)

# Print summary of model layers
model.summary()

Freezing layers

In [ ]:
# Play around with freezing layers, take a look at the tutorial notebook for reference 

Compiling model

In [ ]:
# Compile the model (should be done *after* setting layers to non-trainable)
    # optimizer: adam
    # loss: categorical crossentropy
    # metrics: accuracy
## YOUR CODE HERE

## Train model

In [ ]:
# train the model on the new data for a few epochs
# See how long the model took to train

#This function is called after each epoch
#(It will ensure that your training process does not consume all available RAM)
class garbage_collect_callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    gc.collect()

import time
start_time = time.time()
print('starting model training')
history = model.fit(train_images,
                    train_labels, 
                    batch_size=32, 
                    epochs=40, 
                    verbose=1, 
                    validation_data=(test_images, test_labels),
                    callbacks = [garbage_collect_callback()])
print("--- %s seconds ---" % (time.time() - start_time))

Plot model train/validation accuracy and model train/validation loss

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Make Predictions for Test Images

In [ ]:
# Predict class of test images
predict = model.predict(test_images, verbose=1)

In [ ]:
# Plot a set of test images, along with predicted labels and true labels

print(predict.shape)
id = np.argmax(predict, axis=1)
print(id.shape)
print(id[0])

plt.figure(figsize=(20,20))
for ii in range(0, 16):
    plt.subplot(4,4,ii+1)
    plt.imshow(test_images[ii+100,:,:,:].astype(np.float32))
    plt.axis('off')
    plt.title('expected : ' + label_to_str[np.argmax(test_labels[ii+100])]
              + '\npredicted : ' + label_to_str[id[ii+100]])
plt.show()